# Script to convert generic TIFFs to pyramidal tiled

See key post at https://legacy.imagemagick.org/discourse-server/viewtopic.php?t=20193 for background on using command-line ImageMagick to do the conversion.

In [1]:
import os
import pandas as pd

source_image_root_directory = '/users/baskausj/gallery_digital_image_archive/'
destination_image_root_directory = '/users/baskausj/gallery_pyramidal_tiffs/'

image_data_frame = pd.read_csv('image_dimensions.csv')
# Convert some columns to integers
image_data_frame[['kilobytes', 'height', 'width']] = image_data_frame[['kilobytes', 'height', 'width']].astype(int)

# Clean extensions in rows with alternative extensions
tiff_alt_extensions = ['TIF', 'tiff', 'TIFF']
for tiff_extension in tiff_alt_extensions:
    image_data_frame.loc[image_data_frame.extension == tiff_extension] = 'tif'
    
'''
# For testing purposes, just use the first few rows of the works metadata
test_rows = 4
image_data_frame = image_data_frame.head(test_rows).copy()
'''

image_data_frame.head()


,name,accession,kilobytes,height,width,subdir,extension
0,1956.058.tif,1956.058,197147,6540,10288,1956,tif
1,1956.028.jpg,1956.028,71,428,312,1956,jpg
2,1956.014.jpg,1956.014,261,370,500,1956,jpg
3,1956-035.tif,1956.035,13261,1800,1256,1956,tif
4,1956-034.tif,1956.034,398437,9682,7022,1956,tif


In [2]:
# Filter only rows that are TIFFs
tiffs_frame = image_data_frame.loc[image_data_frame.extension == 'tif']
tiffs_frame.head()

,name,accession,kilobytes,height,width,subdir,extension
0,1956.058.tif,1956.058,197147,6540,10288,1956,tif
3,1956-035.tif,1956.035,13261,1800,1256,1956,tif
4,1956-034.tif,1956.034,398437,9682,7022,1956,tif
10,1956-036.tif,1956.036,159174,6189,4388,1956,tif
11,1956-037.tif,1956.037,28819,1898,2590,1956,tif


In [ ]:
# Basic conversion command is
# os.system('convert 1979.0342P.tif -define tiff:tile-geometry=256x256 -depth 8 ptif:1979.0342P_tiled.tif')

for index, image in tiffs_frame.iterrows():
    in_path = source_image_root_directory + image['subdir'] + '/' + image['name']
    
    # Create the subdirectory if it doesn't already exist
    os.makedirs(destination_image_root_directory + image['subdir'], exist_ok=True)
    out_path = destination_image_root_directory + image['subdir'] + '/' + image['name']
    print(image['name'])
    
    # Redirect (append) errors to a log file
    command_string = 'convert ' + in_path + ' -define tiff:tile-geometry=256x256 -depth 8 ptif:' + out_path + ' 2>> ~/gallery_image_conversion_log.txt'
    #print(command_string)
    os.system(command_string)

print('done')